# 不使用我们框架直接依据规则生成QA

In [7]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
import jsonlines
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time
import re
from tqdm import tqdm

## 主流程

In [8]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"

# 缓存ID，假设从创建缓存的响应中获取
cache_id = "cache-ezjtmpuoc6di11gdsdm1"

# 发送DELETE请求删除缓存
response = requests.delete(
    url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
    headers={
        "Authorization": f"Bearer {api_key}"
    }
)

# 打印响应状态码
print(response.status_code)

# 如果需要，也可以打印响应内容
print(response.text)

200
{"deleted":true,"id":"cache-ezjtmpuoc6di11gdsdm1","object":"cache_object.deleted"}


In [9]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"


# 读取文本文件内容
def load_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


# 创建缓存
def create_cache(file_content):
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/caching",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    cache_response = json.loads(response.text)
    return cache_response['id']


# 检查缓存是否存在且未过期
def check_cache(cache_id):
    response = requests.get(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}"
        }
    )
    if response.status_code == 200:
        return True
    return False


# 重新加载数据并更新缓存
def reload_and_update_cache(file_path, cache_id):
    new_file_content = load_file_content(file_path)
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": new_file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.put(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.status_code == 200


# 使用缓存内容并添加问题
def use_cache_with_question(cache_id, question):
    data = {
        "model": "moonshot-v1-32k",
        "messages": [
            {
                "role": "cache",
                "content": f"cache_id={cache_id};reset_ttl=3600",
            },
            {
                "role": "user",
                "content": question
            }
        ],
        "max_tokens": 8192,
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.json()['choices'][0]['message']['content']

In [11]:
# 主流程
file_path = 'prompts/DISC/根据规则生成答案.txt'
file_content = load_file_content(file_path)
print(f"File{file_path} success read")
cache_id = create_cache(file_content)
print(f"cache id:{cache_id}")
print("cache cuccess")

# # 模拟缓存过期
# time.sleep(3600)

# 检查缓存是否存在且未过期
if not check_cache(cache_id):
    print("检查不到缓存")
    # 重新加载数据并更新缓存
    if reload_and_update_cache(file_path, cache_id):
        print("缓存更新成功")
    else:
        print("缓存更新失败")

print("缓存已存在")

with open('ft_data/RJUA/train_v1.json', 'r', encoding='utf-8') as file:
    answer = json.load(file)

for idx, value in tqdm(enumerate(answer), total=len(answer), desc="Processing"):
    # for num, context in enumerate(answer[idx]["reference"]):
    # 使用缓存内容并添加问题
    context = value["input"]
    question = f"请回答关于文件内容的问题，其中[[CONTEXT]]代表的数据为{context}"
    response = use_cache_with_question(cache_id, question)
    judgement = {
        "id": idx,
        "rule": value["input"],
        "generatrion": response,
        "ori_question": value["instruction"],
        "ori_answer": value["output"]
    }
    # 打开文件以进行写入，如果文件不存在，会创建文件
    with jsonlines.open('data/RJUA/根据规则直接生成_v1.jsonl', mode='a') as writer:
        writer.write(judgement)


Fileprompts/DISC/根据规则生成答案.txt success read
cache id:cache-ezu4975oc6di11ggp811
cache cuccess
缓存已存在


Processing: 100%|██████████| 1642/1642 [1:20:36<00:00,  2.95s/it]
